# Web Scrape labelled images of building styles

The ACOToronto website contains the TOBuilt database -- an open source database of images and information about buildings and structures in toronto.
http://www.acotoronto.ca/tobuilt_new_detailed.php

Order of actions
* we will first use the search tool on the main page to find all the available architectural styles (http://www.acotoronto.ca/tobuilt_new_detailed.php)
* then we will call the page for each style.  This page contains a thumbnail image of each building classified in that style, a link for more details and basic info about the building.  
Note: you generally have to follow a redirection to get to the style page
(http://www.acotoronto.ca/search_buildingsR-d.php?sid=8065)
* download the image locally to image/<style>
* call building details page to get info on building dates, architects etc
(http://www.acotoronto.ca/show_building.php?BuildingID=3883)
 page is structured with alternating building_info and building_info2 divs
building_info contains the name of the info to follow
building_info2 contains the value
The companies row contains the architects. There can be muliple architects for a building, so these are < li > items.
Sometimes the archtect is a hyperlink, but not always, so need to handle both cases specially

American colonial
Annex Style
Art deco
Arts and Crafts
Beaux arts
Brutalist
Byzantine
Chicago style
Classical revival
Commercial style
Contemporary
Deconstructivism
Dutch colonial
Early modern
Edwardian classical
English Cottage style
Georgian revival
Gothic revival
Greek revival
International style
Italianate
Late modernist
Log construction
Mid century expressionist
Mirrored tower
Modern classical
Modern historicist
Modernist
Neo Palladian
Neo-Chateau
Neo-Georgian
Neo-modernist
Neo-Tudor
Postmodern
Prairie style
Queen Anne
Regency
Renaissance revival
Richardsonian Romanesque
Romanesque revival
Sculptural
Second empire
Shingle style
Spanish colonial
Toronto Bay and Gable
Workers Cottage

In [1]:
# Import libaries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import string
import re
import urllib
import os
import time
from datetime import datetime

In [2]:
def load_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Error connecting: status code {response.status_code}")

In [3]:
def download_image(style, img_url):
    '''
    * downloads image to the appropriate style sub-folder in the images directory (and creates folder if missing)
    * to test: download_image('test','/tobuilt_bk/php/Buildingimages/106BedfordRd.jpg')
    '''

    dest_dir = f"../Images/{style}"

    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    full_url = f'{site_root}/{img_url}'
    image_path = f"{dest_dir}/{img_url.split('/')[-1]}"
    res = urllib.request.urlretrieve(full_url, image_path)


#### Building Details pages
* page is structured with alternating building_info and building_info2 divs
* building_info contains the name of the info to follow
* building_info2 contains the value
* The companies row contains the architects.  There can be muliple architects for a building, so these are < li > items.
* Sometimes the archtect is a hyperlink, but not always, so need to handle both cases specially

In [4]:
def get_building_details(bld_link):
    
    flags_dict = {
        'Completed':None,
        'Demolished': None,
        'Companies': None,
        'Type':None,
        'Current use': None,
        'Heritage': None,
       'Notes': None
    }
    architects=[]
    
    full_url = f'{site_root}{bld_link}'
    html_bld = load_page(full_url)
    soup_bld = BeautifulSoup(html_bld)
    # get all building info elements
    info = soup_bld.find_all('div',{'class': 'building_info'})
    for inf in info:
        for flag in flags_dict.keys():
            if flag in inf.text:
                if 'Companies' in flag:
                    company_box = inf.find_next()

                    if 'Architect - ' in company_box.text:
                        li_items = company_box.find_all('li')

                        for arc in li_items:
                            if (arc.find('span')):
                                architect = arc.find('span').previousSibling
                                if architect.name =='a':
                                    architects.append(architect.text.strip())
                                else:
                                    architects.append(architect.replace('-','').strip())
                                
                        flags_dict[flag] = architects
                else:
                    flags_dict[flag] = inf.find_next().text.strip()
    return flags_dict

### Get Buildings for style

In [5]:
def get_buildings_for_style(soup, style):
    buildings=[]
   
    for building in soup.find_all('div',{'class': 'box'}):
        build_style = building.find('div',{'class': 'box_image'})['style']
        image_url = re.findall('\((.*?)\)', build_style)[0]
        build_dict={
            'Style':style,
            'Name':building.findChild('span',{'class':'title'}).text,
            'Bld_link':building.findChild('a').get_attribute_list(key='href')[0],
            'Image':image_url,
            'Address': str(building.findChild('div',{'class':'the_box_text'}).findChild('p')).replace('<br/>',' ').replace('<p>','').replace('</p>','')
        }

        download_image(style,image_url)
        #follow link to get more info on building
        if debug==True: 
            print(build_dict['Bld_link'])
        build_details_dict = get_building_details(build_dict['Bld_link'])
        build_dict = {**build_dict, **build_details_dict}
        buildings.append(build_dict)
   # bld_df = pd.DataFrame(buildings)
    return buildings

In [6]:

def process_style (style):
    print (f"loading {style}")
    curr_style_url = f"{style_url}?MainStyle={style}"
    #print(curr_style_url)
    html = load_page(curr_style_url)
    soup = BeautifulSoup(html)
    if 'document.location' in soup.text:
        # need to redirect to another page
        redirect_url = soup.text.strip().replace('document.location = "','').replace('";',"")
        curr_style_url=f"{site_root}{redirect_url}"
       # print(curr_style_url)
        html2 = load_page(curr_style_url)
        soup = BeautifulSoup(html2)
        
    style_list = get_buildings_for_style(soup, style)
    print(style_list)
    buildings_list.extend(style_list)

### Set up base variables

In [7]:

main_page = 'http://www.acotoronto.ca/tobuilt_new_detailed.php'
style_url="http://www.acotoronto.ca/search_buildingsDB_d2.php"
site_root = "http://www.acotoronto.ca/"
debug=False
buildings_list=[]


In [8]:

html = load_page(main_page)
soup = BeautifulSoup(html)
style_options = soup.find('select',{'name':'MainStyle'}).find_all('option')
for style in style_options[1:]:
    process_style(style.text)
    time.sleep(5)


loading American colonial
[{'Style': 'American colonial', 'Name': 'Robert Grieg House', 'Bld_link': 'show_building.php?BuildingID=4386', 'Image': '/tobuilt_bk/php/Buildingimages/20ChestnutPark.jpg', 'Address': '20 Chestnut Park Rosedale Toronto', 'Completed': '1905', 'Demolished': None, 'Companies': ['Alfred E. Boultbee'], 'Type': 'Detached house', 'Current use': 'Residential', 'Heritage': 'South Rosedale Heritage Conservation District', 'Notes': 'First Occupant: Greig, Robert\r\n\r\nFirst OccupantÂ´s Occupation: Pres The Robert Greig Co. Ltd.'}, {'Style': 'American colonial', 'Name': '22 Chestnut Park', 'Bld_link': 'show_building.php?BuildingID=4388', 'Image': '/tobuilt_bk/php/Buildingimages/22ChestnutPark.jpg', 'Address': '22 Chestnut Park Rosedale Toronto', 'Completed': '1905', 'Demolished': None, 'Companies': ['Alfred E. Boultbee'], 'Type': 'Detached house', 'Current use': 'Residential', 'Heritage': 'South Rosedale Heritage Conservation District', 'Notes': 'First Occupant: Falconbr

loading Art deco
[{'Style': 'Art deco', 'Name': 'The Bentley', 'Bld_link': 'show_building.php?BuildingID=11791', 'Image': '/tobuilt_bk/php/Buildingimages/2e4ae3d2.jpg', 'Address': 'Bentley Apartments 55 Lombard Street  St. Lawrence Toronto', 'Completed': '1925', 'Demolished': None, 'Companies': ['Tanner Hill Designs (Paul Maggiacomo)', 'Ross & MacDonald'], 'Type': None, 'Current use': 'Residential', 'Heritage': 'No heritage status', 'Notes': 'First opened in November 1925, the St. James Auto Parking Garage had space for 500 cars. Built to serve the growing business section and office towers east of Yonge and south of Queen. It was designed by Montreal-based architectural firm Ross & Macdonald who also designed the Maple Leaf Gardens. When prohibition was lifted in 1927, the LCBO Store No. 1 opened on the first floor of the parking garage. They remained here until 1938. The parking garage was converted to condominiums and officially opened on March 17, 1982. The with ramping system rema

loading Arts and Crafts
[{'Style': 'Arts and Crafts', 'Name': '154-156 Amelia Street', 'Bld_link': 'show_building.php?BuildingID=3883', 'Image': '/tobuilt_bk/php/Buildingimages/154-156Amelia.jpg', 'Address': '154-156 Amelia Street Cabbagetown Toronto', 'Completed': 'unknown', 'Demolished': None, 'Companies': None, 'Type': 'Semi-detached house', 'Current use': 'Residential', 'Heritage': 'Cabbagetown North Heritage Conservation District', 'Notes': None}, {'Style': 'Arts and Crafts', 'Name': 'Sunny View Public School', 'Bld_link': 'show_building.php?BuildingID=11142', 'Image': '/tobuilt_bk/php/Buildingimages/1f5e28c0.jpg', 'Address': '450 Blythwood Road Lawrence Park Toronto', 'Completed': '1953', 'Demolished': None, 'Companies': None, 'Type': 'School', 'Current use': 'Educational', 'Heritage': None, 'Notes': None}, {'Style': 'Arts and Crafts', 'Name': "St. Matthew's Lawn Bowling Club", 'Bld_link': 'show_building.php?BuildingID=3458', 'Image': '/tobuilt_bk/php/Buildingimages/StMatthewsLaw

loading Beaux arts
[{'Style': 'Beaux arts', 'Name': 'Ontario Heritage Foundation', 'Bld_link': 'show_building.php?BuildingID=7331', 'Image': '/tobuilt_bk/php/Buildingimages/OntarioHeritageFoundation1.jpg', 'Address': '10 Adelaide Street East Downtown East Toronto', 'Completed': '1908', 'Demolished': None, 'Companies': ['G.W. Gouinlock'], 'Type': 'Mid-rise', 'Current use': 'Office', 'Heritage': 'National Historic Site', 'Notes': 'This building was renovated in 1986-88.'}, {'Style': 'Beaux arts', 'Name': 'York University Glendon Campus; Glendon Hall Gates', 'Bld_link': 'show_building.php?BuildingID=6452', 'Image': '/tobuilt_bk/php/Buildingimages/GlendonHallGates1.jpg', 'Address': '2275 Bayview Avenue Blythwood North York', 'Completed': '1931', 'Demolished': None, 'Companies': ['Molesworth, West & Secord'], 'Type': 'Gates', 'Current use': None, 'Heritage': 'Heritage property', 'Notes': None}, {'Style': 'Beaux arts', 'Name': 'Crescent School', 'Bld_link': 'show_building.php?BuildingID=9195

loading Brutalist
[{'Style': 'Brutalist', 'Name': 'Adanac Apartments', 'Bld_link': 'show_building.php?BuildingID=5', 'Image': '/tobuilt_bk/php/Buildingimages/AdanacApts.jpg', 'Address': '140 Adanac Drive Scarborough Village Scarborough', 'Completed': '1972', 'Demolished': None, 'Companies': None, 'Type': 'High-rise', 'Current use': 'Social housing', 'Heritage': 'No heritage status', 'Notes': None}, {'Style': 'Brutalist', 'Name': 'Toronto Fire Station #332', 'Bld_link': 'show_building.php?BuildingID=3216', 'Image': '/tobuilt_bk/php/Buildingimages/FireStation332.jpg', 'Address': '260 Adelaide Street West Entertainment District Toronto', 'Completed': '1971', 'Demolished': None, 'Companies': None, 'Type': 'Low-rise', 'Current use': 'Fire station', 'Heritage': 'No heritage status', 'Notes': None}, {'Style': 'Brutalist', 'Name': 'Four Seasons Yorkville Hotel', 'Bld_link': 'show_building.php?BuildingID=77', 'Image': '/tobuilt_bk/php/Buildingimages/FourSeasonsYorkvilleHotel.jpg', 'Address': '2

...

loading Sculptural
[{'Style': 'Sculptural', 'Name': 'Giraffe', 'Bld_link': 'show_building.php?BuildingID=8795', 'Image': '/tobuilt_bk/php/Buildingimages/GiraffeR3.jpg', 'Address': '1540 Bloor Street West High Park Toronto', 'Completed': '2011', 'Demolished': None, 'Companies': ['Teeple Architects Inc.'], 'Type': 'High-rise', 'Current use': 'Residential', 'Heritage': 'No heritage status', 'Notes': None}, {'Style': 'Sculptural', 'Name': '1 Bond Avenue', 'Bld_link': 'show_building.php?BuildingID=9662', 'Image': '/tobuilt_bk/php/Buildingimages/1BondAve1.jpg', 'Address': '1 Bond Avenue Don Mills North York', 'Completed': 'unknown', 'Demolished': None, 'Companies': ['Zak Ghanim was born in Egypt and came to Canada in 1975.'], 'Type': 'Detached house', 'Current use': 'Residential', 'Heritage': 'No heritage status', 'Notes': None}, {'Style': 'Sculptural', 'Name': '3 Bond Avenue', 'Bld_link': 'show_building.php?BuildingID=9663', 'Image': '/tobuilt_bk/php/Buildingimages/3BondAve1.jpg', 'Address'

loading Second empire
[{'Style': 'Second empire', 'Name': '100 Adelaide Street East', 'Bld_link': 'show_building.php?BuildingID=8341', 'Image': '/tobuilt_bk/php/Buildingimages/100AdelaideStE.jpg', 'Address': '100 Adelaide Street East Downtown East Toronto', 'Completed': '1889', 'Demolished': None, 'Companies': None, 'Type': 'Mid-rise', 'Current use': 'Commercial', 'Heritage': 'St. Lawrence Neighbourhood Heritage Conservation District', 'Notes': 'This propertyâ\x80\x99s contribution lies in its prominent siting across from St.\r\nJames Park and its historic role in the economic development of the\r\nDistrict in the late 19th century. The structure also contributes to the\r\nphysical character of the District through its tripartite design;\r\nexpressed cornice; mansard roof; storefront with a recessed\r\nentrance; and materiality (buff brickwork) - attributes which\r\ncharacterize the commercial warehouse typology within the District.'}, {'Style': 'Second empire', 'Name': 'Bank of Upper 

loading Shingle style
[{'Style': 'Shingle style', 'Name': '52 Elgin Avenue', 'Bld_link': 'show_building.php?BuildingID=4090', 'Image': '/tobuilt_bk/php/Buildingimages/52ElginAv.jpg', 'Address': '52 Elgin Avenue The Annex Toronto', 'Completed': '1898', 'Demolished': None, 'Companies': ['Frederick H. Herbert'], 'Type': 'Detached house', 'Current use': 'Residential', 'Heritage': 'East Annex Heritage Conservation District', 'Notes': None}, {'Style': 'Shingle style', 'Name': '80 Hogarth Avenue', 'Bld_link': 'show_building.php?BuildingID=9274', 'Image': '/tobuilt_bk/php/Buildingimages/80HogarthAve.jpg', 'Address': '80 Hogarth Avenue Riverdale Toronto', 'Completed': 'unknown', 'Demolished': None, 'Companies': None, 'Type': 'Detached house', 'Current use': 'Residential', 'Heritage': 'No heritage status', 'Notes': None}, {'Style': 'Shingle style', 'Name': '58 Poplar Plains Road', 'Bld_link': 'show_building.php?BuildingID=8674', 'Image': '/tobuilt_bk/php/Buildingimages/58PoplarPlainsRd.jpg', 'Ad

loading Toronto Bay and Gable
[{'Style': 'Toronto Bay and Gable', 'Name': 'Richard Corrigan House / Rock A. Armstrong House', 'Bld_link': 'show_building.php?BuildingID=6552', 'Image': '/tobuilt_bk/php/Buildingimages/9-11AmeliaSt.jpg', 'Address': '9-11 Amelia Street Cabbagetown Toronto', 'Completed': '1896', 'Demolished': None, 'Companies': None, 'Type': 'Semi-detached house', 'Current use': 'Residential', 'Heritage': 'Cabbagetown-Metcalfe Heritage Conservation District', 'Notes': 'No. 9 is known as the Richard Corrigan house; No. 11 is the Rock A. Armstrong House.'}, {'Style': 'Toronto Bay and Gable', 'Name': '14-18 Amelia Street', 'Bld_link': 'show_building.php?BuildingID=3876', 'Image': '/tobuilt_bk/php/Buildingimages/14-18Amelia.jpg', 'Address': '14-18 Amelia Street Cabbagetown Toronto', 'Completed': '1888', 'Demolished': None, 'Companies': None, 'Type': 'Rowhouse', 'Current use': 'Residential', 'Heritage': 'Cabbagetown-Metcalfe Heritage Conservation District', 'Notes': 'No. 14 orig

loading Workers Cottage
[{'Style': 'Workers Cottage', 'Name': 'Angus McBean House / Marmaduke Richardson House', 'Bld_link': 'show_building.php?BuildingID=6553', 'Image': '/tobuilt_bk/php/Buildingimages/15-17AmeliaSt1.jpg', 'Address': '15-17 Amelia Street Cabbagetown Toronto', 'Completed': '1875', 'Demolished': None, 'Companies': None, 'Type': 'Semi-detached house', 'Current use': 'Residential', 'Heritage': 'Cabbagetown-Metcalfe Heritage Conservation District', 'Notes': 'No. 5 is known as the Angus McBean House; No. 17 is the Marmaduke Richardson house.'}, {'Style': 'Workers Cottage', 'Name': 'Amelia Cottages', 'Bld_link': 'show_building.php?BuildingID=3907', 'Image': '/tobuilt_bk/php/Buildingimages/36-40Amelia1.jpg', 'Address': '36-40 Amelia Street Cabbagetown Toronto', 'Completed': '1873', 'Demolished': None, 'Companies': None, 'Type': 'Rowhouse', 'Current use': 'Residential', 'Heritage': 'Cabbagetown-Metcalfe Heritage Conservation District', 'Notes': None}, {'Style': 'Workers Cottag

http://www.acotoronto.ca/search_buildingsDB_d2.php?MainStyle=Postmodern
http://www.acotoronto.ca/search_buildingsDB_d2.php?MainStyle=Arts%20and%20Crafts
http://www.acotoronto.ca/search_buildingsDB_d2.php?MainStyle=Annex%20Style

In [9]:
bld_df = pd.DataFrame(buildings_list)

In [10]:
bld_df.to_csv('../data/aco_buildings_'+ str(round(time.time(),0)) + '.csv')

In [ ]:
#